<a href="https://colab.research.google.com/github/YongWanJin/KDTteam5_semiproject/blob/main/code/%EC%BD%94%EB%A1%9C%EB%82%98%EC%9D%B4%ED%9B%84_%EB%A7%A4%EC%B6%9C%EC%95%A1%ED%9A%8C%EB%B3%B5_%EC%8B%9C%EA%B0%81%ED%99%94(%EB%A6%AC%EB%89%B4%EC%96%BC%2C%20%EB%AF%B8%EC%99%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from glob import glob      
import pandas as pd
from IPython.display import display 

In [ ]:
### 데이터 불러오기

# 1. 상권코드&추정매출액 (2017~2022Q1) 불러오기&병합 : sales
# 2022년 1분기까지 모두 포함

file_names = glob("/content/gdrive/MyDrive/datafile/추정매출(2017~2022Q1)/*.csv") 

sales = pd.DataFrame()    
for file_name in file_names : 
    temp = pd.read_csv(file_name, encoding='CP949')
    sales = pd.concat([sales, temp])               
del(temp)

sales.reset_index(inplace=True, drop=True)          

display(sales)

In [58]:
# 2. 상권코드&행정동코드&좌표값 불러오기 : location
location = pd.read_csv("/content/gdrive/MyDrive/datafile/상권영역_위도경도추가.csv")


# 3. 행정동코드&행정동이름 불러오기 : name
name = pd.read_csv("/content/gdrive/MyDrive/datafile/행정동코드-이름_매치파일.csv")

In [59]:
### sales 전처리 : sales 데이터에서 필요한 부분만 골라내기

# 분기 순서에 맞춰 필요한 연도 추출 : 2019년 3분기 ~ 2022년 1분기
sales = sales[ sales['기준_년_코드'] >= 2019  ] # 연도 추출
sales.drop( sales[(sales['기준_년_코드']==2019)&(sales['기준_분기_코드']<=2)].index, inplace=True) # 필요없는 분기 제거


# 필요한 업종만 추출 : 식당업
sales['서비스_업종_코드'] = sales['서비스_업종_코드'].str.replace(pat=r'[CS]',repl=r'', regex=True).astype('int')
sales = sales[ sales['서비스_업종_코드'] <= 100010]  

# 필요한 columns 골라내기
sales = sales[['기준_년_코드', '기준_분기_코드', '상권_코드', '분기당_매출_금액', '상권_구분_코드_명', '서비스_업종_코드_명']]

display(sales) # 결과

,기준_년_코드,기준_분기_코드,상권_코드,분기당_매출_금액,상권_구분_코드_명,서비스_업종_코드_명
0,2021,1,2110001,626851391,골목상권,한식음식점
1,2021,1,2110001,122957138,골목상권,분식전문점
2,2021,1,2110001,5093349,골목상권,호프-간이주점
3,2021,1,2110001,13507733,골목상권,커피-음료
12,2021,1,2110002,361894416,골목상권,한식음식점
...,...,...,...,...,...,...
825947,2022,1,2110002,346852071,골목상권,한식음식점
825956,2022,1,2110001,20931521,골목상권,커피-음료
825957,2022,1,2110001,5119924,골목상권,호프-간이주점
825958,2022,1,2110001,92420017,골목상권,분식전문점


In [60]:
### name의 상권코드와 location의 행정동코드 병합하기 (location에 병합)

# location 전처리 : 필요한 행만 골라내기
location = location[['상권_코드', '행정동_코드', '경도', '위도']]

# name 전처리 : location의 행정동코드와 자릿수를 같게 만들기
name['행정동코드'] = name['행정동코드']/100
name['행정동코드'] = name['행정동코드'].astype(int)

# name 전처리 : 필요한 열, 필요한 행만 골라내기
name = name[name['시도명']=='서울특별시']
name = name[['행정동코드', '시군구명', '읍면동명']]

# location과 name을 location에 join
location = pd.merge(location, name, left_on='행정동_코드', right_on='행정동코드')
location.drop('행정동_코드', axis='columns', inplace=True) # 중복되는 열 제거

display(location) # 결과

,상권_코드,경도,위도,행정동코드,시군구명,읍면동명
0,2110753,126.908844,37.448687,11545710,금천구,시흥제5동
1,2110748,126.904810,37.448765,11545710,금천구,시흥제5동
2,2110754,126.910150,37.452869,11545710,금천구,시흥제5동
3,2130238,126.910095,37.451617,11545710,금천구,시흥제5동
4,2110427,127.070978,37.634469,11350611,노원구,하계1동
...,...,...,...,...,...,...
3677,2130166,126.918620,37.594260,11380590,은평구,응암제2동
3678,2130113,127.094587,37.606387,11260690,중랑구,신내2동
3679,2130051,127.017283,37.562266,11140665,중구,동화동
3680,2130102,127.062547,37.604597,11230750,동대문구,이문제2동


In [61]:
### location과 sales 병합하기 (sales에 병합)
sales = pd.merge(sales, location, left_on = '상권_코드', right_on='상권_코드')
sales = sales[['기준_년_코드', '기준_분기_코드', '시군구명', '읍면동명', '분기당_매출_금액', '상권_구분_코드_명', '서비스_업종_코드_명', '경도', '위도']]

# 꼬인 순서를 분기 순서에 맞게 재배치
temp1 = sales[sales['기준_년_코드']==2019].sort_values(by='기준_분기_코드')
for y in range(2020, 2023) :
    temp2 = sales[sales['기준_년_코드']==y].sort_values(by='기준_분기_코드')
    temp1 = pd.concat([temp1, temp2])
sales = temp1
del(temp1)
del(temp2)

In [62]:
display(sales)

,기준_년_코드,기준_분기_코드,시군구명,읍면동명,분기당_매출_금액,상권_구분_코드_명,서비스_업종_코드_명,경도,위도
64,2019,3,종로구,평창동,617366771,골목상권,한식음식점,126.956884,37.609345
77952,2019,3,영등포구,양평제2동,1715934320,골목상권,한식음식점,126.892219,37.536260
77951,2019,3,영등포구,양평제2동,1715934320,골목상권,한식음식점,126.892219,37.536260
77950,2019,3,영등포구,양평제2동,1715934320,골목상권,한식음식점,126.892219,37.536260
130882,2019,3,중구,명동,2170985300,발달상권,양식음식점,126.977667,37.567626
...,...,...,...,...,...,...,...,...,...
75476,2022,1,금천구,독산제2동,58629176,골목상권,패스트푸드점,126.901897,37.463747
75475,2022,1,금천구,독산제2동,19662439,골목상권,치킨전문점,126.901897,37.463747
75474,2022,1,금천구,독산제2동,160504059,골목상권,분식전문점,126.901897,37.463747
75551,2022,1,금천구,시흥제1동,199970430,골목상권,분식전문점,126.902834,37.447169


In [71]:
### 연도와 분기를 구분해주는 라벨링 작업 : 추후 반복작업을 위해서
sales_time = sales[['기준_년_코드', '기준_분기_코드']].drop_duplicates()
sales_time['시점_구분_변수'] = [p for p in range(len(sales_time))]
sales = pd.merge(sales, sales_time)

In [93]:
### 구별 전분기 대비 매출액 증가율 집계

# 집계한 결과들을 기입하기 위한 변수 means_gu 생성
# ex) 2019년4분기 : 2019년3분기 대비 2019년4분기의 매출액 증가율
index = [gu for gu in sales['시군구명'].unique()]
columns = ["2019년4분기", "2020년1분기", "2020년2분기", "2020년3분기", "2020년4분기",
           "2021년1분기", "2021년2분기", "2021년3분기", "2021년4분기", "2022년1분기"]
means_gu = pd.DataFrame(index = index, columns = columns)  

# 증가율 집계
for p in range( len(sales['시점_구분_변수'].unique())-1 ) :

    means_before = sales[ sales['시점_구분_변수']==p ].groupby('시군구명')['분기당_매출_금액'].mean()
    means_before = np.array(means_before) # 전분기

    means_now = sales[ sales['시점_구분_변수']==p+1 ].groupby('시군구명')['분기당_매출_금액'].mean()
    means_now = np.array(means_now) # 이번 분기

    inc = (means_now - means_before)/means_before*100 # 전분기 대비 이번 분기 증가율 공식
    inc = np.round(inc, 2).reshape(25,1)
    inc = inc.tolist()
    means_gu.iloc[:,[p]] = inc # 입력

display(means_gu) # 결과



,2019년4분기,2020년1분기,2020년2분기,2020년3분기,2020년4분기,2021년1분기,2021년2분기,2021년3분기,2021년4분기,2022년1분기
종로구,10.78,-18.52,4.96,-2.78,-7.45,-8.26,19.57,-14.81,29.28,-15.09
영등포구,11.18,-13.19,8.0,-8.38,-9.19,-9.08,13.08,-4.87,17.09,-13.25
중구,8.75,-14.27,13.39,-10.06,-14.18,-13.27,22.72,-5.93,12.27,-15.6
강동구,6.1,-15.67,4.85,-4.71,-9.92,-7.67,14.07,-8.87,14.6,-11.82
금천구,3.49,-14.19,5.15,-8.29,-12.27,-11.82,14.53,-7.1,21.4,-13.24
송파구,18.62,-21.56,4.17,-7.74,-8.87,-10.83,22.45,-10.53,20.37,-13.37
강북구,9.12,-15.37,6.6,-5.72,-6.53,-6.48,13.27,-14.6,20.29,-14.98
구로구,8.1,-13.67,9.2,-4.65,-5.82,-7.47,16.84,-10.83,20.44,-14.19
성북구,12.27,-15.06,8.04,-8.31,-9.91,-11.56,20.28,-6.77,12.09,-12.4
동작구,5.93,-10.1,9.77,-8.96,-8.71,-9.58,15.23,-3.09,9.52,-12.18


In [ ]:
### 집계 결과 시각화

# 해야할것들

# 1-1. 2020년 1분기에 타격을 가장 많이 받은 구 순으로 막대그래프

# 1-2. 1-1의 결과를 지도시각화

# 2-1. 코로나 이후 증가율이 회복세에 보이는지 추세 그래프

# 2-2. 그중 회복세가 저조한 지역을 지도 위에 표기
